In [1]:
#Import Dependencies
%matplotlib inline
import os
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import json
from pprint import pprint
from datetime import datetime
from datetime import timedelta 
import time
import matplotlib.pyplot as plt

In [2]:
#!which chromedriver

In [3]:
#Variables
start_date = "2008-01-01"
initial_date = datetime(2008, 1, 1)
end_date = (datetime.today() - timedelta(days = 3)).date()
print(str(initial_date.date()))

2008-01-01


In [5]:
executable_path = {'executable_path': 'C:/Users/joaqu/OneDrive/Curso/Proy_final/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [6]:
#Banxico URL
url = 'https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/anuncios-politica-monetaria-t.html'
browser.visit(url)
api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
links = []
for link in soup.find_all('a'):
    if ('/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria') in link.get('href'):
        partial_link = link.get('href')
        links.append('http://www.banxico.org.mx' + partial_link)
links = links[1:]
#links

In [9]:
dates = []
for date in soup.find_all("td"):
    data = date.get_text()
    dates.append(data)
del dates[1::2]

In [10]:
import re
dates2 = []
for index in range(len(dates)):
    dates2.append(re.sub("\s", "", dates[index])) 


In [11]:
dates3 = []
for index in range(len(dates)):
    dates3.append(re.sub("[\/]", "", dates2[index])) 
#print(dates3)

In [ ]:
name_counter = 0
for i in links:
    r = requests.get(i)
    out_file = open(f'{dates3[name_counter]}.pdf', 'wb')
    out_file.write(r.content)
    name_counter += 1

In [ ]:
#Guillermo 
#pdf to text

In [12]:
final_dates = []
for date in dates2:
    dates = datetime.strptime(date, "%d/%m/%y")
    if dates > initial_date:
        final_dates.append(datetime.strftime(dates, "%Y-%m-%d"))
final_dates.append(datetime.strftime(end_date, "%Y-%m-%d"))

In [13]:
#MXN Lvl
Series = "SF43788"

fxt = []
for fecha in final_dates:
    url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
    for query in url_consulta: 
        data = requests.get(query).json()
        fxt.append(data["bmx"]["series"][0]["datos"][0]["dato"])
time.sleep(180) #tiempo del API

In [14]:
#Tasa Obejtivo
Series = "SF43878"

tasa = []
for fecha in final_dates:
    try:
        url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
        for query in url_consulta: 
            data = requests.get(query).json()
            tasa.append(data["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        tasa.append(0)
time.sleep(180) #tiempo del API   

In [15]:
#Inflacion
Series = "SP30578"

#Total Data
total_query_url = f"{api_url}{Series}/datos/{start_date}/{end_date}?token={banxico_key}"
total_data = requests.get(total_query_url).json()
#print(total_data)
data_total = total_data['bmx']['series'][0]['datos']
data_total

[{'fecha': '01/01/2008', 'dato': '3.70000000'},
 {'fecha': '01/02/2008', 'dato': '3.72000000'},
 {'fecha': '01/03/2008', 'dato': '4.25000000'},
 {'fecha': '01/04/2008', 'dato': '4.55000000'},
 {'fecha': '01/05/2008', 'dato': '4.95000000'},
 {'fecha': '01/06/2008', 'dato': '5.26000000'},
 {'fecha': '01/07/2008', 'dato': '5.39000000'},
 {'fecha': '01/08/2008', 'dato': '5.57000000'},
 {'fecha': '01/09/2008', 'dato': '5.47000000'},
 {'fecha': '01/10/2008', 'dato': '5.78000000'},
 {'fecha': '01/11/2008', 'dato': '6.23000000'},
 {'fecha': '01/12/2008', 'dato': '6.53000000'},
 {'fecha': '01/01/2009', 'dato': '6.28000000'},
 {'fecha': '01/02/2009', 'dato': '6.20000000'},
 {'fecha': '01/03/2009', 'dato': '6.04000000'},
 {'fecha': '01/04/2009', 'dato': '6.17000000'},
 {'fecha': '01/05/2009', 'dato': '5.98000000'},
 {'fecha': '01/06/2009', 'dato': '5.74000000'},
 {'fecha': '01/07/2009', 'dato': '5.44000000'},
 {'fecha': '01/08/2009', 'dato': '5.08000000'},
 {'fecha': '01/09/2009', 'dato': '4.8900

In [16]:
#Match fechas
data_total_df = pd.DataFrame(data_total)
data_total_df['fecha'] = pd.to_datetime(data_total_df['fecha'], format='%d/%m/%Y')
data_total_df['key'] = pd.to_datetime(data_total_df['fecha']).dt.to_period('M')
data_total_df

,fecha,dato,key
0,2008-01-01,3.70000000,2008-01
1,2008-02-01,3.72000000,2008-02
2,2008-03-01,4.25000000,2008-03
3,2008-04-01,4.55000000,2008-04
4,2008-05-01,4.95000000,2008-05
...,...,...,...
143,2019-12-01,2.83000000,2019-12
144,2020-01-01,3.24000000,2020-01
145,2020-02-01,3.70000000,2020-02
146,2020-03-01,3.25000000,2020-03


In [18]:
pre_banxico_dict = {"MXN": fxt,  "Tasa": tasa}
pre_banxico_df = pd.DataFrame(pre_banxico_dict)
pre_banxico_df['Fecha'] = final_dates
pre_banxico_df['key'] = pd.to_datetime(pre_banxico_df['Fecha']).dt.to_period('M')
pre_banxico_df = pre_banxico_df.sort_values('Fecha')
pre_banxico_df = pre_banxico_df[pre_banxico_df.Tasa != 0]
pre_banxico_df = pre_banxico_df.reset_index(drop = True)
pre_banxico_df

,MXN,Tasa,Fecha,key
0,10.1825,8.6600,2008-08-15,2008-08
1,10.6370,8.7500,2008-09-19,2008-09
2,12.7600,8.7350,2008-10-17,2008-10
3,13.3800,8.8500,2008-11-28,2008-11
4,13.9000,8.5650,2009-01-16,2009-01
...,...,...,...,...
99,18.6190,7.3550,2020-02-13,2020-02
100,24.2160,7.0425,2020-03-20,2020-03
101,24.3030,6.5015,2020-04-21,2020-04
102,24.0500,6.0763,2020-05-14,2020-05


In [31]:
banxico_merge = data_total_df.merge(pre_banxico_df, on='key')
banxico_merge['Tasa'] = banxico_merge['Tasa'].astype(float)
banxico_merge['Dif'] = 0
banxico_merge.tail()


,fecha,dato,key,MXN,Tasa,Fecha,Dif
97,2019-11-01,2.97000000,2019-11,19.3760,7.7287,2019-11-14,0
98,2019-12-01,2.83000000,2019-12,18.9480,7.5575,2019-12-19,0
99,2020-02-01,3.70000000,2020-02,18.6190,7.3550,2020-02-13,0
100,2020-03-01,3.25000000,2020-03,24.2160,7.0425,2020-03-20,0
101,2020-04-01,2.15000000,2020-04,24.3030,6.5015,2020-04-21,0


In [32]:
#Fix Array
banxico_merge.rename(columns = {'dato':'Inflacion'}, inplace = True) 
banxico_merge = banxico_merge.reset_index(drop = True)
banxico_merge = banxico_merge.drop(['fecha', 'key'], axis = 1) 
banxico_merge['Tasa'] = banxico_merge['Tasa'].astype(float)
banxico_merge = banxico_merge[['Fecha', 'MXN', 'Inflacion', 'Tasa', 'Dif']]
banxico_merge

,Fecha,MXN,Inflacion,Tasa,Dif
0,2008-08-15,10.1825,5.57000000,8.6600,0
1,2008-09-19,10.6370,5.47000000,8.7500,0
2,2008-10-17,12.7600,5.78000000,8.7350,0
3,2008-11-28,13.3800,6.23000000,8.8500,0
4,2009-01-16,13.9000,6.28000000,8.5650,0
...,...,...,...,...,...
97,2019-11-14,19.3760,2.97000000,7.7287,0
98,2019-12-19,18.9480,2.83000000,7.5575,0
99,2020-02-13,18.6190,3.70000000,7.3550,0
100,2020-03-20,24.2160,3.25000000,7.0425,0


In [34]:
for i in range(1, len(banxico_merge) -1 ):
    banxico_merge.loc[i, 'Dif'] = 25 * round((100 * (banxico_merge.loc[i, 'Tasa'] - banxico_merge.loc[i+1, 'Tasa']))/25)

In [35]:
banxico_merge

,Fecha,MXN,Inflacion,Tasa,Dif
0,2008-08-15,10.1825,5.57000000,8.6600,0.0
1,2008-09-19,10.6370,5.47000000,8.7500,0.0
2,2008-10-17,12.7600,5.78000000,8.7350,-0.0
3,2008-11-28,13.3800,6.23000000,8.8500,25.0
4,2009-01-16,13.9000,6.28000000,8.5650,75.0
...,...,...,...,...,...
97,2019-11-14,19.3760,2.97000000,7.7287,25.0
98,2019-12-19,18.9480,2.83000000,7.5575,25.0
99,2020-02-13,18.6190,3.70000000,7.3550,25.0
100,2020-03-20,24.2160,3.25000000,7.0425,50.0


In [45]:
# Assign X (data) and y (target)
X = banxico_merge.drop(["Dif", "Fecha"], axis=1)
y = banxico_merge["Dif"]
print(X.shape, y.shape)

(102, 3) (102,)


In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [47]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
 classifier.fit(X_train, y_train)

C:\Users\joaqu\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\joaqu\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6842105263157895
Testing Data Score: 0.7692307692307693


In [50]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:50]}")
print(f"First 10 Actual labels: {y_test[:50].tolist()}")

First 10 Predictions:   [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0.]
First 10 Actual labels: [25.0, -0.0, 0.0, 0.0, -0.0, -25.0, -0.0, -0.0, -25.0, -0.0, 25.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -50.0, -25.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0]


In [51]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,-0.0,25.0
1,-0.0,-0.0
2,-0.0,0.0
3,-0.0,0.0
4,-0.0,-0.0
5,-0.0,-25.0
6,-0.0,-0.0
7,-0.0,-0.0
8,-0.0,-25.0
9,-0.0,-0.0
